In [1]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
from MTS import *
from orthogonal_array import *

In [2]:
# ファイル読み込み
IN_FILE = '../data/SkillCraft1_Dataset.csv'
df = pd.read_csv(IN_FILE, header=0, index_col=None, sep=',')
df = df.sample(frac=1, random_state=0) # 行シャッフル
df = df.reset_index(drop=True) # インデックスの更新

In [3]:
df.shape

(3395, 20)

In [4]:
df.head()

,GameID,LeagueIndex,Age,HoursPerWeek,TotalHours,APM,SelectByHotkeys,AssignToHotkeys,UniqueHotkeys,MinimapAttacks,MinimapRightClicks,NumberOfPACs,GapBetweenPACs,ActionLatency,ActionsInPAC,TotalMapExplored,WorkersMade,UniqueUnitsMade,ComplexUnitsMade,ComplexAbilitiesUsed
0,4173,2,16,4,400,107.3424,0.001024,0.000234,0,0.000015,0.000044,0.003293,52.9464,57.9911,5.7956,16,0.000878,4,0.000000,0.000073
1,3651,6,21,28,800,65.3730,0.000566,0.000398,0,0.000084,0.000147,0.002475,54.9402,70.1356,4.7542,14,0.000503,4,0.000000,0.000210
2,3356,3,20,6,50,139.5750,0.003515,0.000670,8,0.000126,0.000190,0.004855,24.2507,48.6146,4.7109,26,0.001820,8,0.000000,0.000025
3,1953,3,16,14,600,102.4686,0.001491,0.000248,3,0.000000,0.000308,0.002697,44.4581,69.7719,6.3202,22,0.001218,6,0.000000,0.000000
4,5403,6,21,8,700,156.4614,0.003755,0.000484,7,0.000165,0.000178,0.004964,24.2674,43.5795,5.1615,18,0.002189,7,0.000064,0.000127


In [5]:
df.describe()

,GameID,LeagueIndex,APM,SelectByHotkeys,AssignToHotkeys,UniqueHotkeys,MinimapAttacks,MinimapRightClicks,NumberOfPACs,GapBetweenPACs,ActionLatency,ActionsInPAC,TotalMapExplored,WorkersMade,UniqueUnitsMade,ComplexUnitsMade,ComplexAbilitiesUsed
count,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000
mean,4805.012371,4.184094,117.046947,0.004299,0.000374,4.364654,0.000098,0.000387,0.003463,40.361562,63.739403,5.272988,22.131664,0.001032,6.534021,0.000059,0.000142
std,2719.944851,1.517327,51.945291,0.005284,0.000225,2.360333,0.000166,0.000377,0.000992,17.153570,19.238869,1.494835,7.431719,0.000519,1.857697,0.000111,0.000265
min,52.000000,1.000000,22.059600,0.000000,0.000000,0.000000,0.000000,0.000000,0.000679,6.666700,24.093600,2.038900,5.000000,0.000077,2.000000,0.000000,0.000000
25%,2464.500000,3.000000,79.900200,0.001258,0.000204,3.000000,0.000000,0.000140,0.002754,28.957750,50.446600,4.272850,17.000000,0.000683,5.000000,0.000000,0.000000
50%,4874.000000,4.000000,108.010200,0.002500,0.000353,4.000000,0.000040,0.000281,0.003395,36.723500,60.931800,5.095500,22.000000,0.000905,6.000000,0.000000,0.000020
75%,7108.500000,5.000000,142.790400,0.005133,0.000499,6.000000,0.000119,0.000514,0.004027,48.290500,73.681300,6.033600,27.000000,0.001259,8.000000,0.000086,0.000181
max,10095.000000,8.000000,389.831400,0.043088,0.001752,10.000000,0.003019,0.004041,0.007971,237.142900,176.372100,18.558100,58.000000,0.005149,13.000000,0.000902,0.003084


In [6]:
(df=='?').sum(axis=0)

GameID                   0
LeagueIndex              0
Age                     55
HoursPerWeek            56
TotalHours              57
APM                      0
SelectByHotkeys          0
AssignToHotkeys          0
UniqueHotkeys            0
MinimapAttacks           0
MinimapRightClicks       0
NumberOfPACs             0
GapBetweenPACs           0
ActionLatency            0
ActionsInPAC             0
TotalMapExplored         0
WorkersMade              0
UniqueUnitsMade          0
ComplexUnitsMade         0
ComplexAbilitiesUsed     0
dtype: int64

In [7]:
# '?'を含む行を削除
df = df[(df=='?').sum(axis=1)==0]

In [8]:
df.LeagueIndex.value_counts()

4    811
5    804
6    621
3    553
2    347
1    167
7     35
Name: LeagueIndex, dtype: int64

In [9]:
# データセットの作成
target_col = 'LeagueIndex'
target = 1
X = df.drop(target_col, axis=1)
y = (df[target_col]==target).astype(np.int32)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=47)

In [10]:
# 直交表の利用
oa = np.reshape(L32, (32, -1)) # 直交表整形
del_cols = range(X.shape[1], oa.shape[1])
oa = np.delete(oa, del_cols, axis=1) # 不要列を削除
sn_ratios = orthogonal_array_test(oa, X_train, y_train)
sn_averages_pos, sn_averages_neg = orthogonal_array_sign(oa, sn_ratios)
print(sn_averages_pos-sn_averages_neg) # 項目ごとのSN比平均差、プラスなら異常検知に関して有意な項目

[ 0.04025844  0.4049412  -0.02389055 -0.48502032  0.08114168 -0.34842389
  0.08943626  0.05602175 -0.39109304 -0.21937601  0.07850668  0.7361288
  0.84909659  0.23924544  0.08480833 -0.04521078  0.09483165 -0.27563635
 -0.30952162]


In [11]:
# 項目選択の結果をもとに異常検知
sign_cols = np.where(sn_averages_pos-sn_averages_neg>=0, 1, 0) # 直交表のテストをもとに有用な変数を選択
arr_cols = list(itertools.compress(X_train.columns, sign_cols))
X_train_ext = X_train[arr_cols]
X_test_ext = X_test[arr_cols]

# MT法による異常度計算
mts = MTS()
mts.fit(X_train_ext[y_train==0])
anomaly_score = mts.predict(X_test_ext)
(break_even_point, score), (coverage, detection) = mts.break_even_point(anomaly_score, y_test)
anomaly_score_abnormal = anomaly_score[y_test==1]

1.114581826131935 0.6885245901639344 0.6878001921229587


In [12]:
# 項目選択せずに異常検知
mts = MTS()
mts.fit(X_train[y_train==0])
anomaly_score = mts.predict(X_test)
(break_even_point, score), (coverage, detection) = mts.break_even_point(anomaly_score, y_test)
anomaly_score_abnormal = anomaly_score[y_test==1]

0.9064510952831832 0.6065573770491803 0.6061479346781941
